In [1]:
from keras.applications import vgg16

Using TensorFlow backend.


In [2]:
model=vgg16.VGG16(weights='imagenet',input_shape=(224, 224, 3), include_top=False)

In [3]:
model.layers[0].input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
from keras.layers import Dense

In [5]:
for layers in model.layers:
    layers.trainable=False
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
top_model=model.output

In [7]:
from keras.layers import Flatten
top_model=Flatten()(top_model)
top_model=Dense(units=512,activation='relu')(top_model)

In [8]:
top_model=Dense(units=256,activation='relu')(top_model)

In [9]:
top_model=Dense(units=1, activation='sigmoid')(top_model)

In [10]:
from keras.models import Model

In [11]:
new_model=Model(input=model.input,outputs=top_model)

C:\Users\gauta\anaconda3\envs\tensor\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [12]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [ ]:
from keras.optimizers import Adam
new_model.compile(loss = 'binary_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])

from keras_preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1.0/255.0,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        'training_set/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        'test_set/',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')
new_model.fit(
        training_set,
        steps_per_epoch=72,
        epochs=1,
        validation_data=test_set,
        validation_steps=10)
# training_set is a folder containg two folders named on two pople names and containg their respective pics
#similiarly test_set

Found 144 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
Epoch 1/1
 3/72 [>.............................] - ETA: 7:44 - loss: 1.4348 - accuracy: 0.5125

In [ ]:
training_set.class_indices

In [ ]:
model.summary()

In [2]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    faces = face_classifier.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y-10:y+h+10, x-10:x+w+10]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    name=""
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (224, 224))
        
        # processing the face to predict:
        face=np.expand_dims(face,axis=0)
        
        # predict the face
        if new_model.predict(face)[0][0] == 0:
            name="name1"
        elif new_model.predict(face)[0][0] == 1:
            name="name2"
        
    # Put count on images and display live count
    cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Face identifier', frame)
        

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cv2.destroyAllWindows()  
cap.release()
print("Collecting Samples Complete")

In [ ]:
cap.release()

In [ ]:
new_model.save("face_recog.h5")

Using TensorFlow backend.


array([[0.]], dtype=float32)